In [1]:
# import lib
import tensorflow as tf
import numpy as np
import sys, os,cv2
from sklearn.utils import shuffle
from scipy.misc import imread,imresize
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from skimage.transform import resize
from imgaug import augmenters as iaa
import imgaug as ia
from scipy.ndimage import zoom

In [2]:
# read all of the data
def read_all_images(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)

        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))

        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        return images
def read_labels(path_to_labels):
    """
    :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
    :return: an array containing the labels
    """
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels
    
train_images = read_all_images("../../../DataSet/STL10/stl10_binary/train_X.bin") / 255.0
train_labels = read_labels    ("../../../DataSet/STL10/stl10_binary/train_Y.bin")
test_images  = read_all_images("../../../DataSet/STL10/stl10_binary/test_X.bin")  / 255.0
test_labels  = read_labels    ("../../../DataSet/STL10/stl10_binary/test_y.bin")

label_encoder= OneHotEncoder(sparse=False,categories='auto')
train_labels = label_encoder.fit_transform(train_labels.reshape((-1,1)))
test_labels  = label_encoder.fit_transform(test_labels.reshape((-1,1)))

In [3]:
# some basic statistic of train and test image // hyper
print(train_images.max((0,1,2)),train_images.min((0,1,2)),train_images.mean((0,1,2)),train_images.std((0,1,2)) )
print(train_images.shape)
print(train_labels.shape)
print(test_images.max((0,1,2)),test_images.min((0,1,2)),test_images.mean((0,1,2)),test_images.std((0,1,2)) )
print(test_images.shape)
print(test_labels.shape)

num_epoch = 50 ; learning_rate = 0.0008; batch_size = 25

[1. 1. 1.] [0. 0. 0.] [0.44671062 0.43980984 0.40664645] [0.26034098 0.25657727 0.27126738]
(5000, 96, 96, 3)
(5000, 10)
[1. 1. 1.] [0. 0. 0.] [0.44723063 0.43964247 0.40495725] [0.2605645  0.25666146 0.26997382]
(8000, 96, 96, 3)
(8000, 10)


In [147]:
# import layers
def tf_softmax(x): return tf.nn.softmax(x)
def tf_elu(x):    return tf.nn.elu(x)
def d_tf_elu(x):  return tf.cast(tf.greater(x,0),tf.float32)  + (tf_elu(tf.cast(tf.less_equal(x,0),tf.float32) * x) + 1.0)
def tf_relu(x):   return tf.nn.relu(x)
def d_tf_relu(x): return tf.cast(tf.greater(x,0),tf.float32)

class CNN():

    def __init__(self,k,inc,out, stddev=0.05,which_reg=0,act=tf_elu,d_act=d_tf_elu):
        self.w          = tf.Variable(tf.random_normal([k,k,inc,out],stddev=stddev,seed=2,dtype=tf.float32))
        self.b          = tf.Variable(tf.zeros(out,dtype=tf.float32))
        self.m,self.v   = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.mb,self.vb = tf.Variable(tf.zeros_like(self.b)),tf.Variable(tf.zeros_like(self.b))
        self.act,self.d_act = act,d_act
        self.which_reg  = which_reg
        
    def getw(self): return [self.w,self.b]

    def feedforward(self,input,stride=1,padding='VALID'):
        self.input  = input
        self.layer  = tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding) + self.b 
        self.layerA = self.act(self.layer)
        return self.layerA

    def backprop(self,gradient,stride=1,padding='VALID'):
        grad_part_1 = gradient
        grad_part_2 = self.d_act(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2
        grad_b      = tf.reduce_mean(grad_middle,(0,1,2))/batch_size
        grad        = tf.nn.conv2d_backprop_filter(input = grad_part_3,filter_sizes = tf.shape(self.w),  out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding) / batch_size
        grad_pass   = tf.nn.conv2d_backprop_input (input_sizes = tf.shape(self.input),filter= self.w,out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding)

        if self.which_reg == 0:   grad = grad
        if self.which_reg == 0.5: grad = grad + lamda * (tf.sqrt(tf.abs(self.w))) * (1.0/tf.sqrt(tf.abs(self.w)+ 10e-5)) * tf.sign(self.w)
        if self.which_reg == 1:   grad = grad + lamda * tf.sign(self.w)
        if self.which_reg == 1.5: grad = grad + lamda * 1.0/(tf.sqrt(tf.square(self.w) + 10e-5)) * self.w
        if self.which_reg == 2:   grad = grad + lamda * (1.0/tf.sqrt(tf.square(tf.abs(self.w))+ 10e-5)) * tf.abs(self.w) * tf.sign(self.w)
        if self.which_reg == 2.5: grad = grad + lamda * 2.0 * self.w
        if self.which_reg == 3:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),3)+ 10e-5,-0.66) * tf.pow(tf.abs(self.w),2) * tf.sign(self.w)
        if self.which_reg == 4:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),4)+ 10e-5,-0.75) * tf.pow(tf.abs(self.w),3) * tf.sign(self.w)

        update_w = []
        
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1) ; v_hat = self.v / (1-beta2)
        adam_middle = m_hat * learning_rate/(tf.sqrt(v_hat) + adam_e)
        update_w.append(tf.assign(self.w,tf.subtract(self.w,adam_middle  )))
        
        update_w.append(tf.assign( self.mb,self.mb*beta1 + (1-beta1) * (grad_b)   ))
        update_w.append(tf.assign( self.vb,self.vb*beta2 + (1-beta2) * (grad_b ** 2)   ))
        m_hatb = self.mb / (1-beta1) ; v_hatb = self.vb / (1-beta2)
        adam_middleb = m_hatb * learning_rate/(tf.sqrt(v_hatb) + adam_e)
        update_w.append(tf.assign(self.b,tf.subtract(self.b,adam_middleb  )))
        
        return grad_pass,update_w
    
# declare layer 
def tf_pca_svd(X,mmax=0.8,mmin=0.0): 
    s,U,V  = tf.svd(X,full_matrices=False)
    smin = tf.reduce_min(s,0)
    smax = tf.reduce_max(s,0)
    ScaledS= (mmax-mmin)*((s-smin)/(smax-smin)) + mmin
    recon_data = U @ tf.diag(s) @ tf.transpose(V) * tf.reduce_mean(tf.abs(V) * ScaledS[None,:],0,keepdims =True)
    return recon_data

In [235]:
# restart the graph 
# sess.close()
# tf.reset_default_graph()

l1 = CNN(3,3,16)
l2 = CNN(3,16,16)
l3 = CNN(3,16,16)

l4 = CNN(3,16,32)
l5 = CNN(3,32,32)
l6 = CNN(3,32,32)

l7 = CNN(1,16,16)
l8 = CNN(1,16,16)
l9 = CNN(1,16,16)

In [236]:
# build graph 

def do(layer9,num=10):
    with tf.device('/cpu:0'):
        s,U,V = tf.svd(layer9)
    S    = tf.matrix_diag(s)
    layer9 = U @ S @ tf.transpose(V,(0,1,3,2))[:,:,:,:num] 
    return layer9

x = tf.placeholder(shape=(batch_size,96,96,3),dtype=tf.float32)
y = tf.placeholder(shape=(batch_size,10),dtype=tf.float32)
is_training = tf.placeholder_with_default(input=False,shape=())

layer1 = l1.feedforward(x,padding='VALID',stride=2)
layer2 = l2.feedforward(layer1,padding='VALID',stride=1)
layer3 = l3.feedforward(layer2,padding='VALID',stride=2)

layer4 = l4.feedforward(layer3,padding='VALID',stride=2)
layer5 = l5.feedforward(layer4,padding='VALID',stride=1)
layer6 = l6.feedforward(layer5,padding='VALID',stride=2)
layer6 = do(layer6,16)

layer7 = l7.feedforward(layer6,padding='VALID')
layer8 = l8.feedforward(layer7,padding='VALID')
layer9 = l9.feedforward(layer8,padding='VALID')
layer9 = do(layer9,10)
final_layer   = tf.reduce_mean(layer9,axis=(1,2))

cost       = tf.nn.softmax_cross_entropy_with_logits_v2(logits=final_layer,labels=y)
auto_train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

final_softmax      = tf_softmax(final_layer)
correct_prediction = tf.equal(tf.argmax(final_softmax, 1), tf.argmax(y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# start the training 
# sess = tf.InteractiveSession()
# sess.run(tf.global_variables_initializer())
from sklearn.utils import shuffle

avg_acc_train = 0
avg_acc_test  = 0
for iter in range(num_epoch*100):
    
    for current_batch_index in range(0,len(train_images),batch_size):
        current_data  = train_images[current_batch_index:current_batch_index+batch_size]
        current_label = train_labels[current_batch_index:current_batch_index+batch_size]
        sess_results  = sess.run([accuracy,auto_train],feed_dict={x:current_data,y:current_label,is_training:True})
        sys.stdout.write('Current Iter : ' + str(iter) + '/'+ str(num_epoch)  + ' batch : ' + str(current_batch_index) + '/'+ str(len(train_images)) + ' acc : ' + str(sess_results[0]) + '\r')
        sys.stdout.flush()
        avg_acc_train = avg_acc_train + sess_results[0]
        
    for current_batch_index in range(0,len(test_images),batch_size):
        current_data  = test_images[current_batch_index:current_batch_index+batch_size]
        current_label = test_labels[current_batch_index:current_batch_index+batch_size]
        sess_results  = sess.run([accuracy],feed_dict={x:current_data,y:current_label,is_training:False})
        sys.stdout.write('Current Iter : ' + str(iter) + '/'+ str(num_epoch)  + ' batch : ' + str(current_batch_index) + '/'+ str(len(test_images)) + ' acc : ' + str(sess_results[0]) + '\r')
        sys.stdout.flush()
        avg_acc_test = avg_acc_test + sess_results[0]        
        
    print("\n Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
    avg_acc_train = 0 
    avg_acc_test  = 0

Current Iter : 0/50 batch : 7975/8000 acc : 0.28
 Current : 0 Acc : 0.4951999970525503 Test Acc : 0.39337499977555124

Current Iter : 1/50 batch : 7975/8000 acc : 0.24
 Current : 1 Acc : 0.4959999965876341 Test Acc : 0.3961250003427267

Current Iter : 2/50 batch : 7975/8000 acc : 0.24
 Current : 2 Acc : 0.501799998357892 Test Acc : 0.39475000023376194

Current Iter : 3/50 batch : 7975/8000 acc : 0.24
 Current : 3 Acc : 0.5095999972522258 Test Acc : 0.38975000029895457

Current Iter : 4/50 batch : 7975/8000 acc : 0.28
 Current : 4 Acc : 0.5183999989926815 Test Acc : 0.39112500003539025

Current Iter : 5/50 batch : 7975/8000 acc : 0.32
 Current : 5 Acc : 0.5177999998629094 Test Acc : 0.3979999992530793

Current Iter : 6/50 batch : 7975/8000 acc : 0.48
 Current : 6 Acc : 0.5267999991774559 Test Acc : 0.39087499980814755

Current Iter : 7/50 batch : 7975/8000 acc : 0.48
 Current : 7 Acc : 0.5293999998271466 Test Acc : 0.39050000039860605

Current Iter : 8/50 batch : 7975/8000 acc : 0.46
 C

Current Iter : 68/50 batch : 7975/8000 acc : 0.22
 Current : 68 Acc : 0.791399998664856 Test Acc : 0.35037500069011

Current Iter : 69/50 batch : 7975/8000 acc : 0.16
 Current : 69 Acc : 0.7885999986529351 Test Acc : 0.3442500009201467

Current Iter : 70/50 batch : 7975/8000 acc : 0.16
 Current : 70 Acc : 0.7896000003814697 Test Acc : 0.34925000136718154

Current Iter : 71/50 batch : 7975/8000 acc : 0.24
 Current : 71 Acc : 0.7826000002026557 Test Acc : 0.35475000003352763

Current Iter : 72/50 batch : 7975/8000 acc : 0.22
 Current : 72 Acc : 0.7843999981880188 Test Acc : 0.3597500000614673

Current Iter : 73/50 batch : 7975/8000 acc : 0.24
 Current : 73 Acc : 0.782599998563528 Test Acc : 0.36225000109989197

Current Iter : 74/50 batch : 7975/8000 acc : 0.16
 Current : 74 Acc : 0.8093999978899956 Test Acc : 0.3610000000335276

Current Iter : 75/50 batch : 7975/8000 acc : 0.24
 Current : 75 Acc : 0.806400001347065 Test Acc : 0.367125000921078

Current Iter : 76/50 batch : 7975/8000 acc 

Current Iter : 136/50 batch : 7975/8000 acc : 0.24
 Current : 136 Acc : 0.939199994802475 Test Acc : 0.34962500056717544

Current Iter : 137/50 batch : 7975/8000 acc : 0.36
 Current : 137 Acc : 0.923799996972084 Test Acc : 0.3567500013159588

Current Iter : 138/50 batch : 7975/8000 acc : 0.48
 Current : 138 Acc : 0.9417999944090844 Test Acc : 0.359125000750646

Current Iter : 139/50 batch : 7975/8000 acc : 0.32
 Current : 139 Acc : 0.922599995136261 Test Acc : 0.3547500005690381

Current Iter : 140/50 batch : 7975/8000 acc : 0.24
 Current : 140 Acc : 0.9403999963402748 Test Acc : 0.3572499996284023

Current Iter : 141/50 batch : 7975/8000 acc : 0.42
 Current : 141 Acc : 0.9533999937772751 Test Acc : 0.3578750010346994

Current Iter : 142/50 batch : 7975/8000 acc : 0.44
 Current : 142 Acc : 0.9425999957323075 Test Acc : 0.3626250008121133

Current Iter : 143/50 batch : 7975/8000 acc : 0.28
 Current : 143 Acc : 0.9365999951958657 Test Acc : 0.36437500033061954

Current Iter : 144/50 batc

Current Iter : 203/50 batch : 7975/8000 acc : 0.16
 Current : 203 Acc : 0.9667999944090844 Test Acc : 0.35512500065378844

Current Iter : 204/50 batch : 7975/8000 acc : 0.32
 Current : 204 Acc : 0.9641999945044517 Test Acc : 0.364875000459142

Current Iter : 205/50 batch : 7975/8000 acc : 0.26
 Current : 205 Acc : 0.966999993622303 Test Acc : 0.35162500124424695

Current Iter : 206/50 batch : 7975/8000 acc : 0.28
 Current : 206 Acc : 0.9379999956488609 Test Acc : 0.36187499985098837

Current Iter : 207/50 batch : 7975/8000 acc : 0.28
 Current : 207 Acc : 0.9435999950766564 Test Acc : 0.3622500002384186

Current Iter : 208/50 batch : 7975/8000 acc : 0.24
 Current : 208 Acc : 0.9671999925374984 Test Acc : 0.3637500000651926

Current Iter : 209/50 batch : 7975/8000 acc : 0.24
 Current : 209 Acc : 0.9597999951243401 Test Acc : 0.3577500006183982

Current Iter : 210/50 batch : 7975/8000 acc : 0.24
 Current : 210 Acc : 0.9469999948143959 Test Acc : 0.37050000042654574

Current Iter : 211/50 

Current Iter : 270/50 batch : 7975/8000 acc : 0.24
 Current : 270 Acc : 0.9815999948978424 Test Acc : 0.35712500028312205

Current Iter : 271/50 batch : 7975/8000 acc : 0.36
 Current : 271 Acc : 0.9839999949932099 Test Acc : 0.366125000664033

Current Iter : 272/50 batch : 7975/8000 acc : 0.24
 Current : 272 Acc : 0.9797999945282936 Test Acc : 0.35737499995157124

Current Iter : 273/50 batch : 7975/8000 acc : 0.36
 Current : 273 Acc : 0.9761999934911728 Test Acc : 0.35924999974668026

Current Iter : 274/50 batch : 7975/8000 acc : 0.24
 Current : 274 Acc : 0.9725999939441681 Test Acc : 0.361375001212582

Current Iter : 275/50 batch : 7975/8000 acc : 0.32
 Current : 275 Acc : 0.9669999954104423 Test Acc : 0.36549999970011415

Current Iter : 276/50 batch : 7975/8000 acc : 0.28
 Current : 276 Acc : 0.9653999951481819 Test Acc : 0.36625000077765435

Current Iter : 277/50 batch : 7975/8000 acc : 0.32
 Current : 277 Acc : 0.96979999512434 Test Acc : 0.3554999995511025

Current Iter : 278/50 b

...

Current Iter : 2490/50 batch : 7975/8000 acc : 0.28
 Current : 2490 Acc : 1.0 Test Acc : 0.37162500103004276

Current Iter : 2491/50 batch : 7975/8000 acc : 0.28
 Current : 2491 Acc : 1.0 Test Acc : 0.37137500108219684

Current Iter : 2492/50 batch : 7975/8000 acc : 0.28
 Current : 2492 Acc : 1.0 Test Acc : 0.37137500108219684

Current Iter : 2493/50 batch : 7975/8000 acc : 0.28
 Current : 2493 Acc : 1.0 Test Acc : 0.37137500098906456

Current Iter : 2494/50 batch : 7975/8000 acc : 0.28
 Current : 2494 Acc : 1.0 Test Acc : 0.37175000086426735

Current Iter : 2495/50 batch : 7975/8000 acc : 0.28
 Current : 2495 Acc : 1.0 Test Acc : 0.37125000115483997

Current Iter : 2496/50 batch : 7975/8000 acc : 0.28
 Current : 2496 Acc : 1.0 Test Acc : 0.3717500011902303

Current Iter : 2497/50 batch : 7975/8000 acc : 0.28
 Current : 2497 Acc : 1.0 Test Acc : 0.3718750011175871

Current Iter : 2498/50 batch : 7975/8000 acc : 0.28
 Current : 2498 Acc : 1.0 Test Acc : 0.37200000123120847

Current Iter

Current Iter : 2565/50 batch : 7975/8000 acc : 0.28
 Current : 2565 Acc : 1.0 Test Acc : 0.37787500049453226

Current Iter : 2566/50 batch : 7975/8000 acc : 0.28
 Current : 2566 Acc : 1.0 Test Acc : 0.3775000004796311

Current Iter : 2567/50 batch : 7975/8000 acc : 0.28
 Current : 2567 Acc : 1.0 Test Acc : 0.3777500006603077

Current Iter : 2568/50 batch : 7975/8000 acc : 0.28
 Current : 2568 Acc : 1.0 Test Acc : 0.377250000811182

Current Iter : 2569/50 batch : 7975/8000 acc : 0.28
 Current : 2569 Acc : 1.0 Test Acc : 0.37687500056345014

Current Iter : 2570/50 batch : 7975/8000 acc : 0.28
 Current : 2570 Acc : 1.0 Test Acc : 0.37712500074412675

Current Iter : 2571/50 batch : 7975/8000 acc : 0.28
 Current : 2571 Acc : 1.0 Test Acc : 0.3768750008428469

Current Iter : 2572/50 batch : 7975/8000 acc : 0.28
 Current : 2572 Acc : 1.0 Test Acc : 0.3766250007553026

Current Iter : 2573/50 batch : 7975/8000 acc : 0.28
 Current : 2573 Acc : 1.0 Test Acc : 0.37575000084470955

Current Iter : 2

Current Iter : 2639/50 batch : 7975/8000 acc : 0.24
 Current : 2639 Acc : 1.0 Test Acc : 0.3748750005615875

Current Iter : 2640/50 batch : 7975/8000 acc : 0.24
 Current : 2640 Acc : 1.0 Test Acc : 0.37475000058766456

Current Iter : 2641/50 batch : 7975/8000 acc : 0.24
 Current : 2641 Acc : 1.0 Test Acc : 0.3747500006807968

Current Iter : 2642/50 batch : 7975/8000 acc : 0.24
 Current : 2642 Acc : 1.0 Test Acc : 0.37450000054668636

Current Iter : 2643/50 batch : 7975/8000 acc : 0.24
 Current : 2643 Acc : 1.0 Test Acc : 0.37437500071246177

Current Iter : 2644/50 batch : 7975/8000 acc : 0.24
 Current : 2644 Acc : 1.0 Test Acc : 0.3733750006416813

Current Iter : 2645/50 batch : 7975/8000 acc : 0.24
 Current : 2645 Acc : 1.0 Test Acc : 0.3735000006621704

Current Iter : 2646/50 batch : 7975/8000 acc : 0.24
 Current : 2646 Acc : 1.0 Test Acc : 0.3730000006733462

Current Iter : 2647/50 batch : 7975/8000 acc : 0.24
 Current : 2647 Acc : 1.0 Test Acc : 0.37225000073667613

Current Iter : 

Current Iter : 2714/50 batch : 7975/8000 acc : 0.28
 Current : 2714 Acc : 1.0 Test Acc : 0.3761250005569309

Current Iter : 2715/50 batch : 7975/8000 acc : 0.28
 Current : 2715 Acc : 1.0 Test Acc : 0.3747500007972121

Current Iter : 2716/50 batch : 7975/8000 acc : 0.28
 Current : 2716 Acc : 1.0 Test Acc : 0.3755000007338822

Current Iter : 2717/50 batch : 7975/8000 acc : 0.28
 Current : 2717 Acc : 1.0 Test Acc : 0.37637500069104135

Current Iter : 2718/50 batch : 7975/8000 acc : 0.28
 Current : 2718 Acc : 1.0 Test Acc : 0.3752500005066395

Current Iter : 2719/50 batch : 7975/8000 acc : 0.28
 Current : 2719 Acc : 1.0 Test Acc : 0.37500000060535965

Current Iter : 2720/50 batch : 7975/8000 acc : 0.28
 Current : 2720 Acc : 1.0 Test Acc : 0.37387500060722234

Current Iter : 2721/50 batch : 7975/8000 acc : 0.28
 Current : 2721 Acc : 1.0 Test Acc : 0.37425000038929285

Current Iter : 2722/50 batch : 7975/8000 acc : 0.28
 Current : 2722 Acc : 1.0 Test Acc : 0.374000000488013

Current Iter : 2

Current Iter : 2788/50 batch : 7975/8000 acc : 0.28
 Current : 2788 Acc : 1.0 Test Acc : 0.3717500003054738

Current Iter : 2789/50 batch : 7975/8000 acc : 0.28
 Current : 2789 Acc : 1.0 Test Acc : 0.3720000003930181

Current Iter : 2790/50 batch : 7975/8000 acc : 0.28
 Current : 2790 Acc : 1.0 Test Acc : 0.37212500018067657

Current Iter : 2791/50 batch : 7975/8000 acc : 0.28
 Current : 2791 Acc : 1.0 Test Acc : 0.37212500018067657

Current Iter : 2792/50 batch : 7975/8000 acc : 0.28
 Current : 2792 Acc : 1.0 Test Acc : 0.3717499999329448

Current Iter : 2793/50 batch : 7975/8000 acc : 0.28
 Current : 2793 Acc : 1.0 Test Acc : 0.3721249999478459

Current Iter : 2794/50 batch : 7975/8000 acc : 0.28
 Current : 2794 Acc : 1.0 Test Acc : 0.3722499998752028

Current Iter : 2795/50 batch : 7975/8000 acc : 0.28
 Current : 2795 Acc : 1.0 Test Acc : 0.3718749998603016

Current Iter : 2796/50 batch : 7975/8000 acc : 0.28
 Current : 2796 Acc : 1.0 Test Acc : 0.37149999984540044

Current Iter : 2

In [68]:
# build graph 
x = tf.placeholder(shape=(batch_size,96,96,3),dtype=tf.float32)
y = tf.placeholder(shape=(batch_size,10),dtype=tf.float32)
is_training = tf.placeholder_with_default(input=False,shape=())

layer1 = l1.feedforward(x,padding='VALID',stride=2)
layer2 = l2.feedforward(layer1,padding='VALID',stride=2)
layer3 = l3.feedforward(layer2,padding='VALID',stride=2)

layer4 = l4.feedforward(layer3,padding='VALID',stride=2)
layer5 = l5.feedforward(layer4,padding='VALID')
layer6 = l6.feedforward(layer5,padding='VALID',stride=2)

layer7 = l7.feedforward(layer6,padding='VALID')
layer8 = l8.feedforward(layer7,padding='VALID')
layer9 = l9.feedforward(layer8,padding='VALID')
final_layer   = tf.reduce_mean(layer9,axis=(1,2))

cost       = tf.nn.softmax_cross_entropy_with_logits_v2(logits=final_layer,labels=y)
auto_train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

final_softmax      = tf_softmax(final_layer)
correct_prediction = tf.equal(tf.argmax(final_softmax, 1), tf.argmax(y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [69]:
# start the training 
# sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

avg_acc_train = 0
avg_acc_test  = 0
for iter in range(num_epoch):
    
    train_images,train_labels = shuffle(train_images,train_labels)
    test_images,test_labels   = shuffle(test_images,test_labels)
    
    for current_batch_index in range(0,len(train_images),batch_size):
        current_data  = train_images[current_batch_index:current_batch_index+batch_size]
        current_label = train_labels[current_batch_index:current_batch_index+batch_size]
        sess_results  = sess.run([accuracy,auto_train],feed_dict={x:current_data,y:current_label,is_training:True})
        sys.stdout.write('Current Iter : ' + str(iter) + '/'+ str(num_epoch)  +
                         ' batch : ' + str(current_batch_index) + '/'+ str(len(train_images)) + 
                         ' acc : ' + str(sess_results[0]) + '\r')
        sys.stdout.flush()
        avg_acc_train = avg_acc_train + sess_results[0]
        
    for current_batch_index in range(0,len(test_images),batch_size):
        current_data  = test_images[current_batch_index:current_batch_index+batch_size]
        current_label = test_labels[current_batch_index:current_batch_index+batch_size]
        sess_results  = sess.run([accuracy],feed_dict={x:current_data,y:current_label,is_training:False})
        sys.stdout.write('Current Iter : ' + str(iter) + '/'+ str(num_epoch)  +
                         ' batch : ' + str(current_batch_index) + '/'+ str(len(test_images)) + 
                         ' acc : ' + str(sess_results[0]) + '\r')
        sys.stdout.flush()
        avg_acc_test = avg_acc_test + sess_results[0]        
        
    print("\n Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
    avg_acc_train = 0 
    avg_acc_test  = 0

Current Iter : 0/50 batch : 7975/8000 acc : 0.28
 Current : 0 Acc : 0.18139999855309724 Test Acc : 0.23037499892525376

Current Iter : 1/50 batch : 7975/8000 acc : 0.32
 Current : 1 Acc : 0.2577999993786216 Test Acc : 0.29962500096298755

Current Iter : 2/50 batch : 7975/8000 acc : 0.24
 Current : 2 Acc : 0.3040000008791685 Test Acc : 0.29412499961908906

Current Iter : 3/50 batch : 7975/8000 acc : 0.48
 Current : 3 Acc : 0.33220000080764295 Test Acc : 0.34125000082422047

Current Iter : 4/50 batch : 7975/8000 acc : 0.48
 Current : 4 Acc : 0.37060000136494636 Test Acc : 0.36575000116135925

Current Iter : 5/50 batch : 7975/8000 acc : 0.48
 Current : 5 Acc : 0.39500000055879353 Test Acc : 0.3934999986551702

Current Iter : 6/50 batch : 7975/8000 acc : 0.52
 Current : 6 Acc : 0.4213999977707863 Test Acc : 0.3979999993694946

Current Iter : 7/50 batch : 7975/8000 acc : 0.68
 Current : 7 Acc : 0.4451999981701374 Test Acc : 0.4063749992288649

Current Iter : 8/50 batch : 7975/8000 acc : 0.5

# Reference 
1. Brownlee, J. (2017). How to One Hot Encode Sequence Data in Python. Machine Learning Mastery. Retrieved 9 December 2018, from https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
2. tf.placeholder_with_default | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/placeholder_with_default
3. tf.nn.softmax_cross_entropy_with_logits | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits
4. line, O. (2018). Output without new line. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/2623470/output-without-new-line
5. shell?, H. (2018). How to tell if tensorflow is using gpu acceleration from inside python shell?. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/38009682/how-to-tell-if-tensorflow-is-using-gpu-acceleration-from-inside-python-shell
6. GPU?, H. (2018). How to use TensorFlow GPU?. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/51306862/how-to-use-tensorflow-gpu
7. Batch normalization: theory and how to use it with Tensorflow. (2018). Towards Data Science. Retrieved 9 December 2018, from https://towardsdatascience.com/batch-normalization-theory-and-how-to-use-it-with-tensorflow-1892ca0173ad
8. Batch normalization: theory and how to use it with Tensorflow. (2018). Towards Data Science. Retrieved 9 December 2018, from https://towardsdatascience.com/batch-normalization-theory-and-how-to-use-it-with-tensorflow-1892ca0173ad
9. tf.reset_default_graph | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/reset_default_graph
10. tf.Session | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/Session
11. tf.nn.moments | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/nn/moments
12. CMD?, H. (2018). How do I run two commands in one line in Windows CMD?. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/8055371/how-do-i-run-two-commands-in-one-line-in-windows-cmd
13. loop, B. (2018). Batch script loop. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/2591758/batch-script-loop
14. tf.train.MomentumOptimizer | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/train/MomentumOptimizer
15. Test if two numpy arrays are (close to) equal, i. (2018). Test if two numpy arrays are (close to) equal, including shape. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/32874840/test-if-two-numpy-arrays-are-close-to-equal-including-shape
16. tf.linalg.diag | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/linalg/diag


In [ ]:
temp   = np.random.randn(100,32,32,3)
s,U,V  = tf.svd(temp)
S = tf.matrix_diag(s)
print(S)
print(U,s,V)
print(tf.diag(s))

result = U @ S @ tf.transpose(V,(0,1,3,2))
result = result.eval()

print(np.testing.assert_allclose(temp, result))
print(np.allclose(temp,result))
print(np.equal(temp,result).sum())
print(100*32*32*3)